In [37]:
# 1.Write a program that demonstrates exception propagation across multiple functions.

def function_c():
    raise ZeroDivisionError("Division by zero occurred in function_c()")

def function_b():
    function_c()

def function_a():
    function_b()

def main():
    try:
        function_a()
    except ZeroDivisionError as e:
        print("Exception caught in main():",type(e).__name__,  "-", e)
        
main()


Exception caught in main(): ZeroDivisionError - Division by zero occurred in function_c()


In [38]:
# 2.Write a program using exception chaining (raise ... from ...).

class InvalidAmountError(Exception):
    pass

def parse_amount(s):
    try:
        return float(s)
    except ValueError as e:
        # chain the low-level ValueError into a domain-specific exception
        raise InvalidAmountError(f"Cannot parse amount from '{s}'") from e

try:
    amt = parse_amount("12.8.99") 
except InvalidAmountError as e:
    print("Chained exception:", e)
    # show original exception via __cause__
    print("Original cause:", type(e.__cause__).__name__, "-", e.__cause__)

Chained exception: Cannot parse amount from '12.8.99'
Original cause: ValueError - could not convert string to float: '12.8.99'


In [39]:
# 3.Write a program that simulates database connection and raises custom exceptions.

class DatabaseConnectionError(Exception):
    pass

class QueryExecutionError(Exception):
    pass

try:
    connected = False  # simulate DB is not connected
    if not connected:
        raise DatabaseConnectionError("ERROR! Unable to connect to DB")
    query = "DROP TABLE users"
    if "DROP" in query.upper():
        raise QueryExecutionError("ERROR! Dangerous query detected")
    print("Query executed successfully!")

except (DatabaseConnectionError, QueryExecutionError) as e:
    print("DB error:", e)

DB error: ERROR! Unable to connect to DB


In [40]:
# 4.Write a program that reads from a file using with statement and handles errors.

filename = "data.txt"
try:
    with open(filename, "r", encoding="utf-8") as f:
        for i, line in enumerate(f, 1):
            print(i, line.strip())
except FileNotFoundError:
    print("File not found:", filename)
except UnicodeDecodeError:
    print("Could not decode file (wrong encoding).")
except Exception as e:
    print("Unexpected error while reading file:", type(e).__name__, e)

File not found: data.txt


In [ ]:
# 5.Write a program that skips bad data rows in a CSV file using exception handling.

import csv


good_rows = []
bad_count = 0
try:
    with open("people.csv", newline="") as f:
        reader =csv.DictReader(f)
        for row in reader:
            try:
                if not row["name"].strip():
                    raise ValueError("Name is empty")
                row["age"] = int(row["age"])
                row["salary"] = float(row["salary"])
                good_rows.append(row)
            except Exception:
                bad_count += 1
                print("Skipping bad row:", row)
except FileNotFoundError:
    print("ERROR! Csv file not found")

print("Good rows:", good_rows)
print("Skipped rows:", bad_count)

ERROR! Csv file not found
Good rows: []
Skipped rows: 0


In [42]:

# 6. Write a program that handles timeout error when connecting to a server (use requests).

import requests

url = "https://httpbin.org/delay/5"  # endpoint that delays response for demo

try:
    # timeout can be a single float (seconds) or a tuple (connect_timeout, read_timeout)

    resp = requests.get(url, timeout=0.5)  # half a second timeout
    resp.raise_for_status()  # raise HTTPError for 4xx/5xx
    print("Response length:", len(resp.text))

except requests.Timeout:
    print("Request timed out (server too slow).")

except requests.HTTPError as e:
    print("HTTP error:", e)

except requests.RequestException as e:
    print("Other requests error:", e)

Request timed out (server too slow).


In [43]:
# 7.Write a program that handles exceptions in a recursive factorial function.

def factorial(n):
    if not isinstance(n, int):
        raise TypeError("factorial expects an integer")
    if n < 0:
        raise ValueError("factorial not defined for negative numbers")
    if n == 0:
        return 1
    return n * factorial(n - 1)

try:
    print("factorial(5) =", factorial(5))   # 120
    print("factorial(0) =", factorial(0))   # 1
    print("factorial(7) =", factorial(7))   # 5040
    print("factorial(-2) =", factorial(-2))  # raises ValueError
except Exception as e:
    print("Error:", e)

try:
    print("factorial(3.5) =", factorial(3.5))  # raises TypeError
except Exception as e:
    print("Error:", e)



factorial(5) = 120
factorial(0) = 1
factorial(7) = 5040
Error: factorial not defined for negative numbers
Error: factorial expects an integer


In [49]:
# 8.Write a program that simulates transaction rollback in a banking system using exceptions.

class TransactionError(Exception):
    pass
# Initial account balance
balance = 5000
print("balance at start:", balance)
# Take a "snapshot" of balance before transaction
snapshot = balance
try:
    # Start transaction
    # withdraw 80
    balance -= 800  
    print("During transaction, Balance:", balance)
    # Simulate a problem
    raise TransactionError("Something went wrong during withdrawal!") 
except TransactionError as e:
    # Rollback to snapshot if error occurs
    print("Error occurred:", e)
    # rollback
    balance = snapshot  
    print("Transaction rolled back")
print("After:", balance)

balance at start: 5000
During transaction, Balance: 4200
Error occurred: Something went wrong during withdrawal!
Transaction rolled back
After: 5000


In [68]:
# 9.Write a program to validate credit card numbers and raise exceptions for invalid inputs.

class InvalidCardError(Exception):
    pass

def validate_card(number):
    # check digits only
    if not number.isdigit():
        raise InvalidCardError("Card must contain only digits.")
    # check length
    if len(number) not in (12, 13, 16, 20):
        raise InvalidCardError("Invalid card length.")
cards = ["39781234567892340", "3572089392958"]
for card in cards:
    try:
        validate_card(card)
        print(card, "is valid!")
    except InvalidCardError as e:
        print(card, "is invalid:", e)

39781234567892340 is invalid: Invalid card length.
3572089392958 is valid!


In [83]:
# 10.Write a program that raises an exception if memory usage goes beyond a threshold.
import tracemalloc
import logging

# Configure logging
logging.basicConfig(level=logging.WARNING, format="%(levelname)s: %(message)s")

THRESHOLD_MB = 10  # set small for demo

# start tracking memory
tracemalloc.start()

# simulate memory usage by creating a large list
data = [0] * 5_0000_000

# get current and peak memory usage (in bytes)
current, peak = tracemalloc.get_traced_memory()
current_mb = peak / (1024 * 1024)

print(f"Current memory usage: {current_mb:.2f} MB (threshold {THRESHOLD_MB} MB)")

if current_mb > THRESHOLD_MB:
    logging.warning(
        f"Memory usage high: {current_mb:.2f} MB exceeded {THRESHOLD_MB} MB")
else:
    logging.info("Memory usage is within safe limits.")

# stop tracking
tracemalloc.stop()


Current memory usage: 381.49 MB (threshold 10 MB)
